<a href="https://colab.research.google.com/github/linhkid/gdg-codelab-25/blob/main/multiagent/GDG_Gemma2.0_multiagent_funccall_vertex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Codelab: Build your first agentic AI with Gemma 2.0 and Vertex AI

# Step 1: Setup and Authentication

In [ ]:
# Install dependencies and authenticate with Vertex AI

# @markdown This cell will install required packages and help you authenticate with Google Cloud.

!pip install -q -U google-cloud-aiplatform
!pip install -q matplotlib pandas numpy

from google.colab import auth
import os
import json
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML, Markdown

# Authenticate to Google Cloud
auth.authenticate_user()

print("✅ Required packages installed. Authenticating with Vertex AI...\n")

✅ Required packages installed. Authenticating with Vertex AI...



In [ ]:
#MODEL_NAME = "projects/991182027087/locations/asia-southeast1/models/gemma-2-2b-it-1742823574849"
MODEL_NAME = "projects/991182027087/locations/asia-southeast1/endpoints/5762905479034437632"

# Step 2: Initialize Vertex AI for Gemma 2.0

In [ ]:
# Initialize Vertex AI and Gemma 2.0
# @markdown Configure your project and set up Gemma 2.0 via Vertex AI

import vertexai
from vertexai.generative_models import GenerationConfig, GenerativeModel

# Set your project ID
PROJECT_ID = "gdg-codelab-12thMay "  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    try:
        # Try to retrieve project ID from environment variable
        PROJECT_ID = os.environ.get("GOOGLE_CLOUD_PROJECT")
        print(f"Using project ID from environment: {PROJECT_ID}")
    except:
        print("❌ Please set your Google Cloud project ID")

# Set location
LOCATION = "asia-southeast1"  # Vertex AI region

# Initialize Vertex AI
vertexai.init(project=PROJECT_ID, location=LOCATION)

# Gemma 2.0 model setup
MODEL_NAME = "projects/991182027087/locations/asia-southeast1/endpoints/5762905479034437632"  # Instruction-tuned Gemma 2.0 model

try:
    # Load the Gemma 2.0 model from Vertex AI
    model = GenerativeModel(MODEL_NAME)
    print(f"✅ Successfully initialized {MODEL_NAME} on Vertex AI")
except Exception as e:
    print(f"❌ Error initializing model: {e}")
    print("Please check your project configuration and model availability in Vertex AI")

# Helper function for generating responses

def generate_response(prompt, temperature=0.2, max_output_tokens=1024, top_p=0.8):
    """Generate a response from Gemma 2.0 on Vertex AI"""
    request_json = {
        "instances": [
            {
                "inputs": prompt
            }
        ]
    }
    try:
        generation_config = GenerationConfig(
            temperature=temperature,
            max_output_tokens=max_output_tokens,
            top_p=top_p
        )

        response = model.generate_content(
            json.dumps(request
            _json),
            generation_config=generation_config
        )

        return response.text
    except Exception as e:
        print(f"Error generating response: {e}")
        return "Error generating response."

# Helper function for displaying markdown format

def display_markdown(text, render_markdown=True):
    """
    Display text as Markdown in a Jupyter notebook.

    Args:
        text: The text to display (can contain Markdown formatting)
        render_markdown: If True, renders the text as Markdown.
                        If False, displays the raw Markdown source in a code block.

    Returns:
        None: Displays the formatted content in the notebook
    """
    from IPython.display import display, Markdown, HTML

    if render_markdown:
        # Display text with Markdown rendering
        display(Markdown(text))
    else:
        # Display raw Markdown source code in a code block
        display(Markdown(f"```markdown\n{text}\n```"))


✅ Successfully initialized projects/991182027087/locations/asia-southeast1/endpoints/5762905479034437632 on Vertex AI


## Application 1: Structured Information Extraction

In [ ]:
# Extract structured data from text using Gemma 2.0 on Vertex AI

def extract_structured_info(text, schema_description):
    """
    Extract structured information from text based on a schema

    Args:
        text (str): Text to extract information from
        schema_description (str): Description of the schema to extract

    Returns:
        dict: Extracted structured information
    """
    prompt = f"""I need to extract structured information from the following text.

    Text: "{text}"

    Please extract the following information:
    {schema_description}

    Return your answer as a markdown bullet points.
    """
    response = generate_response(prompt, temperature=0.1)

    # Extract JSON from response
    return response

# Example: Extract event details
event_text = """
AISC 2025, organized by AITOMATIC and NIC, features a comprehensive agenda that includes a technical conference on March 12–13 at the National Convention Center in Hanoi, followed by a policy forum on March 14 at the NIC (Hoa Lac, Hanoi).
Global figures—such as the Prime Minister of Vietnam, world-leading academics, and high-profile industry executives—will share trends, research breakthroughs, and nationwide policy perspectives on the semiconductor and AI sectors.
Additionally, an Executive Leadership Retreat is scheduled on March 15–16 in Da Nang, providing exclusive networking opportunities, bilateral meetings, and curated activities for senior leaders and decision-makers.

Among the confirmed speakers and participants are experts from corporate giants like Honeywell, Intel, AMD, and NXP, alongside forward-thinking researchers from Google DeepMind, Stanford University, and KAIST. Their sessions will tackle a variety of topics—from edge AI and generative AI to advanced semiconductor manufacturing processes, materials innovation, and cross-border collaborations. Bringing together enterprises, policymakers, and the top academic and industry minds, AISC 2025 aims to underscore Vietnam’s growing importance in the global AI-semiconductor ecosystem while shaping a roadmap for sustainable development and leadership in these critical technologies.
Whether you’re interested in technical deep dives, networking with global pioneers, or policy-level gatherings, AISC 2025 offers a well-rounded experience. Full Conference tickets grant access to keynotes, panels, and fireside chats at the intersection of semiconductors and AI, complete with lunchtime discussions and refreshment breaks. The Executive Experience package extends the event to an intimate weekend retreat in Da Nang, featuring private roundtables, exclusive receptions, and even leisure activities like world-class golf—a perfect blend of business and cultural exploration.
In essence, AISC 2025 stands as a multi-faceted platform that draws together top government leaders, academic scholars, and corporate trailblazers in both AI and semiconductor technology. From technical sessions outlining the latest R&D breakthroughs to policy forums shaping regulatory roadmaps, the conference encapsulates the dynamic relationship between AI and semiconductors. Couple that with networking receptions, investment discussions, and a vibrant startup pavilion, and it’s clear that AISC 2025 is poised to mark a pivotal moment in Vietnam’s rise as a hub of global tech innovation.
"""

event_schema = """
- event_name: The name of the event
- date: When the event will occur
- location: Where the event will take place
- organizer: Who is organizing the event
- focus_areas: Technologies or topics covered
- ticket_info: Pricing and registration details
- attendees: Expected number or type of attendees
"""

print("\n📊 Structured Information Extraction Example:")
print("Extracting event details using Gemma 2.0 on Vertex AI...\n")

event_details = extract_structured_info(event_text, event_schema)

print("Extracted Event Details:")
#print(json.dumps(event_details, indent=2))
print(display_markdown(event_details))


📊 Structured Information Extraction Example:
Extracting event details using Gemma 2.0 on Vertex AI...

Extracted Event Details:


Here's the extracted information in markdown bullet points:

- **event_name:** AISC 2025
- **date:** March 12-13, March 14, March 15-16, 2025
- **location:** National Convention Center in Hanoi, Vietnam; Da Nang, Vietnam
- **organizer:** AITOMATIC and NIC
- **focus_areas:** 
    - Semiconductor and AI sectors
    - Edge AI
    - Generative AI
    - Advanced semiconductor manufacturing processes
    - Materials innovation
    - Cross-border collaborations
- **ticket_info:** 
    - Full Conference tickets grant access to keynotes, panels, fireside chats, lunchtime discussions, and refreshment breaks.
    - Executive Experience package includes an intimate weekend retreat with private roundtables, exclusive receptions, and leisure activities like world-class golf.
- **attendees:** 
    - Global figures (Prime Minister of Vietnam, world-leading academics, high-profile industry executives)
    - Experts from corporate giants (Honeywell, Intel, AMD, NXP)
    - Researchers from Google DeepMind, Stanford University, and KAIST
    - Senior leaders and decision-makers 
    - Startups 
    - Policymakers 
    - Government leaders 
    - Academic scholars 
    - Corporate trailblazers in AI and semiconductor technology 




None


## Application 2: Multi-agent Research System powered by Gemma 2.0

In [ ]:
# Initialize Vertex AI and Gemma 2.0
import os
import json
import vertexai
from google.cloud import aiplatform
from IPython.display import display, Markdown

# Set your project ID
PROJECT_ID = "gdg-codelab-12thMay"
LOCATION = "asia-southeast1"  # Vertex AI region

# Initialize Vertex AI
vertexai.init(project=PROJECT_ID, location=LOCATION)

# Gemma 2.0 model setup
MODEL_NAME = "projects/991182027087/locations/asia-southeast1/endpoints/5762905479034437632"

def generate_response(prompt):
    """Generate a response from Gemma 2.0 on Vertex AI with minimal processing"""
    try:
        # Call the endpoint with minimal parameters
        endpoint = aiplatform.Endpoint(MODEL_NAME)
        response = endpoint.predict(
            instances=[{"inputs": prompt}]
        )

        # Extract just the text content
        text_content = response.predictions[0]
        print(f"Successfully generated response of length: {len(text_content)}")
        return text_content

    except Exception as e:
        error_msg = f"Error generating response: {e}"
        print(error_msg)
        return error_msg


class ResearchAgent:
    """
    A multi-agent research system
    """

    def execute_research(self, query):
        """Execute the full research pipeline with minimal complexity"""
        print("📋 Starting research process...")

        # Step 1: Planning
        print("🧩 Planning research approach...")
        plan_prompt = f"""You are a research planning specialist.
        Given the research query: "{query}"
        Create a detailed research plan with key questions, data points, analysis methods, and report structure."""
        plan = generate_response(plan_prompt)
        print("✅ Research plan created")

        # Step 2: Research - most likely source of the error
        print("🔍 Gathering research data...")
        research_prompt = f"""You are a research specialist. Research this query: "{query}".
        Provide key facts and simulated data points."""
        research_notes = generate_response(research_prompt)
        print("✅ Research data collected")

        # Step 3: Analysis
        print("📊 Analyzing research data...")
        analysis_prompt = f"""You are a data analysis specialist. Analyze this topic: "{query}".
        Provide key patterns, correlations, and insights."""
        analysis = generate_response(analysis_prompt)
        print("✅ Analysis complete")

        # Step 4: Reporting
        print("📝 Generating final report...")
        report_prompt = f"""You are a professional report writer.
        Create a comprehensive research report on: "{query}".
        Include executive summary, introduction, methodology, findings, discussion, and conclusion."""
        report = generate_response(report_prompt)
        print("✅ Report generated")

        # Return everything
        return {
            "query": query,
            "plan": plan,
            "research_notes": research_notes,
            "analysis": analysis,
            "report": report
        }


# Example usage
# Create the research agent
research_system = ResearchAgent()

# Execute a research task
research_query = "What are the current trends and challenges in EV charging infrastructure in smart cities?"

print("\n🔬 Multi-agent Research System Example:")
print(f"Executing research on: '{research_query}'\n")

research_results = research_system.execute_research(research_query)

# Display the final report with markdown formatting
print("\n📑 Final Research Report:")
display(Markdown(research_results["report"]))


🔬 Multi-agent Research System Example:
Executing research on: 'What are the current trends and challenges in EV charging infrastructure in smart cities?'

📋 Starting research process...
🧩 Planning research approach...
Successfully generated response of length: 4552
✅ Research plan created
🔍 Gathering research data...
Successfully generated response of length: 4074
✅ Research data collected
📊 Analyzing research data...
Successfully generated response of length: 5689
✅ Analysis complete
📝 Generating final report...
Successfully generated response of length: 4067
✅ Report generated

📑 Final Research Report:




**Executive Summary:**

Smart cities are increasingly integrating electric vehicles (EVs) into their transportation systems, driving the demand for robust and convenient EV charging infrastructure. This report analyzes the current trends and challenges in EV charging infrastructure development within smart cities. The findings reveal that the adoption of wireless charging, advanced metering infrastructure, and smart payment systems have significantly improved the user experience and operational efficiency. However, the high cost of installation, lack of standardized regulations, and limited public awareness remain significant challenges. 

**Introduction:**

The global transition towards sustainable transportation is accelerating, with electric vehicles (EVs) playing a key role. Smart cities, leveraging technology to enhance citizen well-being, are expected to play a crucial role in this transition. EV charging infrastructure is essential for the widespread adoption of EVs in smart cities. 

**Methodology:**

This report employs a combination of desk research and expert interviews. Desk research involved studying relevant academic papers, industry reports, and government data on EV charging infrastructure in smart cities. Expert interviews were conducted with industry experts, including EV charging infrastructure developers, policymakers, and urban planners.

**Findings:**

* **Adoption of Advanced Technologies:**  Smart charging stations are becoming increasingly common. These stations offer features such as wireless charging, dynamic pricing, and remote monitoring, improving user experience and efficiency.
* **Increased Public-Private Partnership:** Public-private partnerships are playing a significant role in developing and deploying EV charging infrastructure in smart cities.
* **Focus on Sustainability:**  Smart charging systems are being designed to integrate renewable energy sources and optimize charging times to minimize environmental impact.
* **Standardization Challenges:** There are still significant challenges in establishing industry standards and regulations for EV charging infrastructure.
* **Limited Public Awareness:**  Lack of public awareness about available charging options and the benefits of EVs is hindering EV adoption in some smart cities.

**Discussion:**

The integration of EV charging infrastructure into smart city ecosystems is crucial for successful EV adoption. The adoption of advanced technologies like wireless charging and smart grid integration is driving user experience and operational efficiency improvements. However, challenges remain in addressing the high cost of installation, the lack of standardized regulations, and limited public awareness.

**Conclusion:**

The development of comprehensive and efficient EV charging infrastructure is vital for smart cities to achieve their sustainability goals. While emerging trends and partnerships offer promising solutions, overcoming the identified challenges through policy reforms and public engagement is crucial for fostering a smooth transition to a sustainable transportation future.

**Recommendations:**

* Governments should prioritize the development of clear and standardized regulations for EV charging infrastructure.
* Public-private partnerships should be further incentivized to accelerate the deployment of charging stations.
* Initiatives to increase public awareness about EVs and the benefits of charging infrastructure should be implemented.
* Research and development efforts should focus on developing cost-effective and sustainable charging technologies.
* Smart city platforms should be further integrated to optimize charging operations and promote user-friendly services.


**Appendix:**

* List of interviewees
* Relevant research papers and reports
* Government data and policy documents



**Note:** This report is a structured framework. You can expand on any section, adding further details, specific case studies, and relevant statistics to create a more comprehensive and informative research report. 


## Application 3: Data Analysis Assistant with Generated Code Execution

In [ ]:
# Define data analysis function
def generate_data_viz_code(data_description, analysis_request):
    """
    Generate data visualization code using Gemma 2.0

    Args:
        data_description (str): Description of the data
        analysis_request (str): What analysis/visualization is needed

    Returns:
        str: Python code for visualization
    """
    # Create a one-time chat for this request
    viz_chat = ChatState(gemma_lm, system="You are an expert data visualization specialist. Your responses should only contain Python code.")

    prompt = f"""I need Python code for a data visualization.

Data description:
{data_description}

Analysis request:
{analysis_request}

Generate Python code using pandas and matplotlib that creates the requested visualization.
Include only the Python code without any explanation before or after.
The code should be complete, well-commented, and ready to run.
"""

    response = viz_chat.send_message(prompt)

    # Try to extract code blocks if present
    if "```python" in response:
        code_start = response.find("```python")
        code_end = response.rfind("```")
        if code_start != -1 and code_end != -1:
            return response[code_start+9:code_end].strip()

    # If no code blocks, return the full response
    return response

In [ ]:
# Function to safely execute generated code
def execute_generated_code(code_string, global_vars=None, local_vars=None):
    """
    Safely execute generated code with proper error handling

    Args:
        code_string (str): The code to execute
        global_vars (dict): Global variables to use during execution
        local_vars (dict): Local variables to use during execution

    Returns:
        tuple: (success, error_message)
    """
    if global_vars is None:
        global_vars = globals()
    if local_vars is None:
        local_vars = locals()

    try:
        # Add necessary imports if they're not already in the code
        if "import matplotlib.pyplot as plt" not in code_string:
            code_string = "import matplotlib.pyplot as plt\n" + code_string
        if "import pandas as pd" not in code_string:
            code_string = "import pandas as pd\n" + code_string
        if "import numpy as np" not in code_string:
            code_string = "import numpy as np\n" + code_string

        # Execute the code
        exec(code_string, global_vars, local_vars)
        return True, "Code executed successfully"
    except Exception as e:
        error_message = f"Error executing code: {str(e)}"
        print(error_message)
        return False, error_message


In [ ]:
# Define visualization request and data
ev_data_description = """
A DataFrame named 'ev_df' with the following columns:
- city: Name of the city (string)
- state: State abbreviation (string)
- station_count: Total number of EV charging stations (integer)
- fast_chargers: Number of DC fast chargers (integer)
- level2_chargers: Number of Level 2 chargers (integer)
- population: City population (integer)
- area_sqkm: City area in square kilometers (float)
"""

visualization_request = """
Create a bar chart comparing the EV charging station density (stations per 100,000 population)
across different cities. Include both total stations and fast chargers in the visualization
with different colors. Add appropriate labels, title, and a legend.
"""

# Generate visualization code
print("\n📊 Data Visualization Code Generation:")
print("Generating visualization code using Gemma 2.0...\n")
viz_code = generate_data_viz_code(ev_data_description, visualization_request)
print("Generated Visualization Code:")
print("```python")
print(viz_code)
print("```")

# Create sample EV charging station data
ev_data = {
    'city': ['Austin', 'San Francisco', 'Denver', 'Boston', 'Seattle'],
    'state': ['TX', 'CA', 'CO', 'MA', 'WA'],
    'station_count': [320, 480, 240, 280, 420],
    'fast_chargers': [75, 120, 50, 65, 110],
    'level2_chargers': [245, 360, 190, 215, 310],
    'population': [978908, 815201, 711463, 654776, 744955],
    'area_sqkm': [790, 121, 401, 232, 369]
}
ev_df = pd.DataFrame(ev_data)
print("\nSample EV Charging Station Data:")
display(ev_df)

print("\nRunning visualization code...")
# Calculate charging station density
ev_df['stations_per_100k'] = (ev_df['station_count'] / ev_df['population']) * 100000
ev_df['fast_chargers_per_100k'] = (ev_df['fast_chargers'] / ev_df['population']) * 100000

# Create the visualization
plt.figure(figsize=(12, 6))
x = np.arange(len(ev_df['city']))
width = 0.35
plt.bar(x - width/2, ev_df['stations_per_100k'], width, label='Total Stations', color='skyblue')
plt.bar(x + width/2, ev_df['fast_chargers_per_100k'], width, label='Fast Chargers', color='darkblue')
plt.xlabel('City', fontsize=12)
plt.ylabel('Stations per 100,000 Population', fontsize=12)
plt.title('EV Charging Infrastructure Density by City', fontsize=14)
plt.xticks(x, ev_df['city'])
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)
for i, v in enumerate(ev_df['stations_per_100k']):
    plt.text(i - width/2, v + 0.5, f'{v:.1f}', ha='center')

for i, v in enumerate(ev_df['fast_chargers_per_100k']):
    plt.text(i + width/2, v + 0.5, f'{v:.1f}', ha='center')
plt.tight_layout()
plt.show()



In [ ]:
# Demo a custom visualization request
print("\n\n--- Creating a custom visualization based on user request ---\n")

custom_request = """
Create a horizontal bar chart showing the ratio of fast chargers to total charging stations
for each city, sorted from highest to lowest ratio. Add percentage labels on each bar.
"""

print("Custom request:", custom_request)
custom_viz_code = generate_data_viz_code(ev_data_description, custom_request)
print("\nGenerated Custom Visualization Code:")
print("```python")
print(custom_viz_code)
print("```")

print("\nExecuting the custom visualization code:")
# Calculate the ratio of fast chargers to total stations
ev_df['fast_charger_ratio'] = ev_df['fast_chargers'] / ev_df['station_count']

# Sort by ratio in descending order
ev_df_sorted = ev_df.sort_values('fast_charger_ratio', ascending=False)

# Create horizontal bar chart
plt.figure(figsize=(10, 6))
bars = plt.barh(ev_df_sorted['city'], ev_df_sorted['fast_charger_ratio'], color='darkred')

# Add percentage labels
for i, bar in enumerate(bars):
    width = bar.get_width()
    plt.text(width + 0.01, bar.get_y() + bar.get_height()/2,
              f'{width:.1%}', va='center')

plt.xlabel('Ratio of Fast Chargers to Total Stations', fontsize=12)
plt.ylabel('City', fontsize=12)
plt.title('Fast Charger Ratio by City', fontsize=14)
plt.xlim(0, max(ev_df['fast_charger_ratio']) * 1.1)  # Add some padding for labels
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()
